In [7]:
from vit_encoder import ViT
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from utils.util import count_model_params, train_epoch,eval_model,train_model
import os
import shutil
from torch.utils.tensorboard import SummaryWriter
from loader.Dataset import VideoDataset 
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

data_path='/home/nfs/inf6/data/datasets/MOVi/movi_c/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
number_of_frames_per_video=24
max_objects_in_scene=11
batch_size=64
img_height=128
img_width=128
channels=3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
#train_dataset=VideoDataset(data_path,split='train',number_of_frames_per_video=number_of_frames_per_video,max_objects_in_scene=max_objects_in_scene) 
#train_loader = DataLoader(dataset=train_dataset,
#                            batch_size=batch_size,
#                            shuffle=True)


test_dataset=VideoDataset(data_path,split='validation',number_of_frames_per_video=number_of_frames_per_video,max_objects_in_scene=max_objects_in_scene)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False) 

coms,bboxs,masks,rgbs,flows=next(iter(test_loader))
print(f"Shapes: >>>>>>>>>>>>>>>>> \r\n{coms.shape=}, \r\n{bboxs.shape=}, \r\n{masks.shape=}, \r\n{rgbs.shape=}, \r\n{flows.shape=}\r\n<<<<<<<<<<<<<<<<<<")

Data Loaded Successfully: len(self.coord_addresses)=250, len(self.mask_addresses)=250, len(self.rgb_addresses)=6000, len(self.flow_addresses)=6000
Shapes: >>>>>>>>>>>>>>>>> 
coms.shape=torch.Size([64, 24, 11, 2]), 
bboxs.shape=torch.Size([64, 24, 11, 4]), 
masks.shape=torch.Size([64, 24, 128, 128]), 
rgbs.shape=torch.Size([64, 24, 3, 128, 128]), 
flows.shape=torch.Size([64, 24, 3, 128, 128])
<<<<<<<<<<<<<<<<<<


In [11]:
vit = ViT(
        img_height=img_height,
        img_width=img_width,
        channels=channels,
        frame_numbers=number_of_frames_per_video,
        token_dim=128,
        attn_dim=128,
        num_heads=4,
        mlp_size=512,
        num_tf_layers=4).to(device)
print(f"ViT has {count_model_params(vit)} parameters")
vit

ViT has 7181056 parameters


ViT(
  (patch_projection): Sequential(
    (0): LayerNorm((49152,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=49152, out_features=128, bias=True)
  )
  (pos_emb): PositionalEncoding()
  (encoderBlocks): Sequential(
    (0): EncoderBlock(
      (ln_att): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
      (attn): MultiHeadAttention(
        (q): Linear(in_features=128, out_features=128, bias=False)
        (k): Linear(in_features=128, out_features=128, bias=False)
        (v): Linear(in_features=128, out_features=128, bias=False)
        (out_proj): Linear(in_features=128, out_features=128, bias=False)
      )
      (ln_mlp): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
      (mlp): MLP(
        (mlp): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=512, out_features=128, bias=True)
        )
      )
    )
    (1): EncoderBlock(
      (ln_att):

In [12]:
!nvidia-smi
#!kill -9 2418075

Thu Sep 25 11:41:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.158.01             Driver Version: 570.158.01     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:01:00.0 Off |                  N/A |
| 82%   88C    P2            324W /  350W |   23661MiB /  24576MiB |    100%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
with torch.no_grad():
    y = vit(rgbs,masks=masks,max_objects_in_scene=11)
attn_maps = vit.get_attn_mask()
print(f"Input Shape: {rgbs.shape}")
print(f"Output Shape: {y.shape}")
print(f"Found {len(attn_maps)} Attn Masps of shape {attn_maps[0].shape}")

Input Shape: torch.Size([64, 24, 3, 128, 128])
Output Shape: torch.Size([64, 24, 11, 128])
Found 4 Attn Masps of shape torch.Size([1536, 4, 11, 11])


In [6]:
# classification loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(vit.parameters(), lr=3e-4)

# Decay LR by a factor of 3 every 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=1/3)

TBOARD_LOGS = os.path.join(os.getcwd(), "tboard_logs", "ViT_30")
if not os.path.exists(TBOARD_LOGS):
    os.makedirs(TBOARD_LOGS)

shutil.rmtree(TBOARD_LOGS)
writer = SummaryWriter(TBOARD_LOGS)

In [ ]:
train_loss, val_loss, loss_iters, valid_acc = train_model(
        model=vit,
        optimizer=optimizer,
        scheduler=scheduler,
        criterion=criterion,
        train_loader=train_loader,
        valid_loader=test_loader,
        num_epochs=30,
        tboard=writer
    )